# Dataset part again

In [ ]:
# Data loading and processing 
import pandas as pd 
import numpy as np
import json
whole_df = pd.read_csv("C:/Users/Sharafat/Desktop/Masters' Dissertation/Structured_whole_dataframe.csv")
whole_df = whole_df.drop("Unnamed: 0", axis = 1)
train_df = whole_df[whole_df["source_x"].astype(str).str.startswith("DDICorpus/Train/")]
train_df
train_df["type"] = train_df["type"].replace("No interaction", "no interaction")
train_df
test_df = whole_df[whole_df["source_x"].astype(str).str.startswith("DDICorpus/Test/")]
test_df
test_df["type"] = test_df["type"].replace("No interaction", "no interaction")
test_df
# example => train_df or test_df 
train_df_records = train_df.to_dict(orient = "records")
train_df_records
test_df_records = test_df.to_dict(orient = "records")
test_df_records
# 1. Sample 10 examples of 'no interaction'
no_interaction_df = train_df[train_df['type'] == 'no interaction'].sample(n=10, random_state=42)

# 2. Sample 3 examples from each of the *other* types (mechanism, effect, etc.)
other_types_df = (
    train_df[train_df['type'] != 'no interaction']
    .groupby('type', group_keys=False)
    .sample(n=3, random_state=42)
)

# 3. Combine and shuffle
examples_df = pd.concat([no_interaction_df, other_types_df]).sample(frac=1, random_state=42).reset_index(drop=True)

examples_df["type"] = examples_df["type"].replace("No interaction", "no interaction")
examples_df
example_shots = examples_df.to_dict(orient="records")
example_shots
train_df_grouped = train_df.groupby("sentence_id")
train_df_grouped
group = train_df_grouped.get_group("DDI-DrugBank.d64.s87")
group
train_df["triplet"] = list(zip(train_df["e1_text"], train_df["type"], train_df["e2_text"]))
train_df
import json

# Group by sentence (assuming sentence text is unique identifier)
grouped = train_df.groupby("sentence_text")

# Build JSON-like structure
json_data = []

for sentence_text, group in grouped:
    # Get triplets from the group
    triplets = list(zip(group["e1_text"], group["type"], group["e2_text"]))
    
    # Optional: if 'type4' column exists, get the most frequent or first type4
    type4_value = group["type"].iloc[0]
    
    # Create record
    json_data.append({
        "sentence": sentence_text,
        "interaction_type": type4_value,
        "triplets": triplets
    })

# Save to file
with open("train_df_sentence_triplets.json", "w") as f:
    json.dump(json_data, f, indent=2)

with open("train_df_sentence_triplets.json","r") as f:
    train_triplets = json.load(f)

train_triplets[0]
import json

# Group by sentence (assuming sentence text is unique identifier)
grouped = test_df.groupby("sentence_text")

# Build JSON-like structure
json_data = []

for sentence_text, group in grouped:
    # Get triplets from the group
    triplets = list(zip(group["e1_text"], group["type"], group["e2_text"]))
    
    # Optional: if 'type4' column exists, get the most frequent or first type4
    type4_value = group["type"].iloc[0]
    
    # Create record
    json_data.append({
        "sentence": sentence_text,
        "interaction_type": type4_value,
        "triplets": triplets
    })

# Save to file
with open("test_df_sentence_triplets.json", "w") as f:
    json.dump(json_data, f, indent=2)
with open("test_df_sentence_triplets.json","r") as f:
    test_triplets = json.load(f)

test_triplets[0]
import json

# Group by unique sentence (or sentence_id if preferred)
grouped = train_df.groupby("sentence_text")

# Build JSON-like structure
sentence_entities = []

for sentence_text, group in grouped:
    # Collect unique entities from both drug1 and drug2
    entities = set(group["e1_text"]).union(set(group["e2_text"]))
    
    sentence_entities.append({
        "sentence": sentence_text,
        "entities": sorted(entities)  # optional: sort for consistency
    })

# Save to JSON
with open("train_df_sentence_entities.json", "w") as f:
    json.dump(sentence_entities, f, indent=2)

with open("train_df_sentence_entities.json","r") as f:
    train_entities = json.load(f)

train_entities[0]
train_triplets
few_shot_examples = train_triplets
few_shot_examples
import json

# Group by unique sentence (or sentence_id if preferred)
grouped = test_df.groupby("sentence_text")

# Build JSON-like structure
sentence_entities = []

for sentence_text, group in grouped:
    # Collect unique entities from both drug1 and drug2
    entities = set(group["e1_text"]).union(set(group["e2_text"]))
    
    sentence_entities.append({
        "sentence": sentence_text,
        "entities": sorted(entities)  # optional: sort for consistency
    })

# Save to JSON
with open("test_df_sentence_entities.json", "w") as f:
    json.dump(sentence_entities, f, indent=2)

with open("test_df_sentence_entities.json","r") as f:
    test_entities = json.load(f)

test_entities[0]
len(train_entities)
len(test_entities)
examples_df
import json

# Group by sentence (assuming sentence text is unique identifier)
grouped = examples_df.groupby("sentence_text")

# Build JSON-like structure
json_data = []

for sentence_text, group in grouped:
    # Get triplets from the group
    triplets = list(zip(group["e1_text"], group["type"], group["e2_text"]))
    
    # Optional: if 'type4' column exists, get the most frequent or first type4
    type4_value = group["type"].iloc[0]
    
    # Create record
    json_data.append({
        "sentence": sentence_text,
        "interaction_type": type4_value,
        "triplets": triplets
    })

# Save to file
with open("few_shot_example_triplets.json", "w") as f:
    json.dump(json_data, f, indent=2)

with open("few_shot_example_triplets.json","r") as f:
    few_shot_entities = json.load(f)

few_shot_entities[0]
len(few_shot_entities)
train_entities
train_triplets # ground truth, remove interaction_type and choose random examples for few_shot
def transform_structure(data):
    output = {'sentence': data['sentence']}
    for idx, entity in enumerate(data['entities'], 1):
        output[f'Entity_{idx}'] = entity
    return output

transformed = transform_structure(train_entities[0])
print(transformed)

def transform_structure(data):
    output = {'Sentence': data['sentence']}
    for idx, entity in enumerate(data['entities'], 1):
        output[f'Entity_{idx}'] = entity
    return output

updated_train_entities = []
for i in range(len(train_entities)):
    transformed = transform_structure(train_entities[i])
    updated_train_entities.append(transformed)
print(updated_train_entities[-1])
len(updated_train_entities)

# Sentence Level Baseline Model

In [ ]:
import os

GEMINI_API_KEY = ''
assert GEMINI_API_KEY, ""

# Set your Gemini API key
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY



from google import genai

client = genai.Client()
    
llm_zero_shot_responses = []
for i in range(len(updated_train_entities)):  # updated_train_entities file is given in both github and zip file
  query = f"""You are a biomedical expert. Such sentence and all drug names in the sentence are given: {updated_train_entities[i]}.

Interaction types are: 

"mechanism": "Biological process causing interaction (e.g., enzyme inhibition)",
    "effect": "Clinical outcome (e.g., increased drug concentration)",
    "advise": "Recommendation (e.g., avoid concomitant use)",
    "int": "General interaction mention without specifics",
    "no interaction": "No explicit or implicit sign of interaction"

Find the interaction type between all possible pairs of these entities and extract triplets in such format: (entity_1, interaction_type, entity_2)

Answer with only triplets
Triplets:  """
  response = client.models.generate_content(
  model="gemma-3-27b-it",
  contents=query,
  config=genai.types.GenerateContentConfig(
      temperature = 0
    
  )
)
  time.sleep(3.5)
  llm_zero_shot_responses.append((updated_train_entities[i], response.text))

  with open("llm_triplet_zero_shot_responses.json","w") as f:
    json.dump(f"Result_{i}: updated_train_entities[i], response.text", f , indent = 2)
  print(f"Predicting {i}")
  print(response.text)

In [ ]:
import json

# Your model predictions: list of (sentence_dict, prediction_text)

# Convert prediction text into structured triplets
def parse_triplets(text):
    triplet_lines = text.strip().split("\n")
    triplets = []
    for line in triplet_lines:
        if line.startswith("(") and line.endswith(")"):
            try:
                parts = line.strip("()").split(", ")
                if len(parts) == 3:
                    triplets.append([parts[0], parts[1], parts[2]])
            except Exception:
                continue
    return triplets

# Merge predictions with ground truth
merged_results = []
for pred in llm_zero_shot_responses:
    sentence_data, prediction_text = pred
    sentence = sentence_data['Sentence']
    predicted_triplets = parse_triplets(prediction_text)

    # Find matching ground truth entry
    gt_entry = next((item for item in train_triplets if item['sentence'].strip() == sentence.strip()), None)
    ground_truth_triplets = gt_entry['triplets'] if gt_entry else []

    merged_results.append({
        "sentence": sentence,
        "ground_truth_triplets": ground_truth_triplets,
        "predicted_triplets": predicted_triplets
    })

# Save merged results
with open("merged_predictions_vs_ground_truth.json", "w") as f:
    json.dump(merged_results, f, indent=2)


In [ ]:
import random

def create_few_shot_prompt(train_triplets, k=5, seed=42):
    random.seed(seed)
    selected = random.sample(train_triplets, k)

    examples = []
    for ex in selected:
        sentence = ex["sentence"].strip()
        triplets = ex.get("triplets", [])

        # Get unique entities from the triplets
        entity_set = set()
        for t in triplets:
            entity_set.add(t[0])
            entity_set.add(t[2])
        entities = ", ".join(sorted(entity_set))

        triplet_lines = "\n".join([f"({t[0]}, {t[1]}, {t[2]})" for t in triplets])

        examples.append(
            f"""Example:
Sentence: "{sentence}"
Entities: {entities}
Triplets:
{triplet_lines}
"""
        )

    return "\n".join(examples)

few_shot_prompt_prefix = create_few_shot_prompt(train_triplets, k=5)

# Then use in your prompting like:
few_shot_prompt = f"{few_shot_prompt_prefix}"
print(few_shot_prompt)

In [ ]:
import os
import json
import time
from google import genai


GEMINI_API_KEY = '' # it is sensitive informative, that is why I removed it
assert GEMINI_API_KEY, ""
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
client = genai.Client()

# Output holder
all_responses = []

# Prompt styles
interaction_types = """Interaction types are:
"mechanism": "Biological process causing interaction (e.g., enzyme inhibition)",
"effect": "Clinical outcome (e.g., increased drug concentration)",
"advise": "Recommendation (e.g., avoid concomitant use)",
"int": "General interaction mention without specifics",
"no interaction": "No explicit or implicit sign of interaction"
"""

few_shot_examples = f"""
{interaction_types}
"""

cot_instruction = f"""
Let's think step by step. First identify all unique entity pairs. Then, analyze the sentence for any interaction clues using the definitions below. Finally, write each result in the format: (entity_1, interaction_type, entity_2)

{interaction_types}
"""

def extract_entities(entity_dict):
    return [v for k, v in entity_dict.items() if k.startswith("Entity_")]

# Inference loop
for i, example in enumerate(updated_train_entities):
    sentence = example["Sentence"]
    entities = extract_entities(example)

    output_entry = {
        "index": i,
        "sentence": sentence,
        "entities": entities
    }

    # Prepare prompts
    zero_shot_prompt = f"""You are a biomedical expert.

Sentence: "{sentence}"
Entities: {", ".join(entities)}

{interaction_types}

Find the interaction type between all possible pairs of these entities and extract triplets in this format:
(entity_1, interaction_type, entity_2)

Answer with only triplets. Do not include any additional text.
Triplets:"""

    few_shot_prompt = f"""{few_shot_examples}

Sentence: "{sentence}"
Entities: {", ".join(entities)}

Answer with only triplets. Do not include any additional text.
Triplets:"""

    cot_prompt = f"""You are a biomedical expert.

Sentence: "{sentence}"
Entities: {", ".join(entities)}

{cot_instruction}

Answer with only triplets. Do not include any additional text.
Triplets:"""

    prompts = {
        "zero_shot_output": zero_shot_prompt,
        "few_shot_output": few_shot_prompt,
        "cot_output": cot_prompt
    }

    for method_name, prompt in prompts.items():
        try:
            response = client.models.generate_content(
                model="gemma-3-27b-it",
                contents=prompt,
                config=genai.types.GenerateContentConfig(
                    temperature=0.0
                )
            )
            time.sleep(3.5)

            output_entry[method_name] = response.text.strip()
            print(f"[{method_name}] Finished index {i}")
            print(f"{method_name}: {response.text.strip()}")

        except Exception as e:
            print(f"Error at index {i} with method {method_name}: {e}")
            output_entry[method_name] = "ERROR"

    all_responses.append(output_entry)

    # Save after each example (optional safety)
    with open("triplet_predictions_all_methods.json", "w") as f:
        json.dump(all_responses, f, indent=2)


# Pair Level Model

In [ ]:
import itertools
import pandas as pd
import numpy as np


# Create expanded pairwise dataset
pairwise_data = []

for idx, item in enumerate(train_triplets):
    sentence = item['sentence']
    triplets = item['triplets']
    
    # Extract all unique entities from the triplets
    all_entities = set()
    for t in triplets:
        all_entities.add(t[0])
        all_entities.add(t[2])
    
    # Generate all possible entity pairs
    entity_pairs = list(itertools.combinations(all_entities, 2))
    
    # Create ground truth mapping for quick lookup
    gt_map = {}
    for t in triplets:
        key1 = (t[0], t[2])
        key2 = (t[2], t[0])
        gt_map[key1] = t[1]
        gt_map[key2] = t[1]  # Relations are bidirectional
    
    # Create entries for each pair
    for e1, e2 in entity_pairs:
        # Determine ground truth
        gt = gt_map.get((e1, e2), "no interaction")
        
        # For explicit "no interaction" in ground truth
        if (e1, e2) in gt_map and gt_map[(e1, e2)] == "no interaction":
            gt = "no interaction"
        
        pairwise_data.append({
            "sentence_id": idx,
            "sentence": sentence,
            "entity1": e1,
            "entity2": e2,
            "ground_truth": gt
        })

# Convert to DataFrame
pairwise_df = pd.DataFrame(pairwise_data)
pairwise_df

In [ ]:
pairwise_df_records = pairwise_df.to_dict(orient="records")
pairwise_df_records

In [ ]:
import json 

with open("pairwise_predictions.json", "r") as f :
    all_responses = json.load(f)
all_responses

# Few-shot random examples selection

In [ ]:
import random

def create_few_shot_prompt(train_triplets, k=2, seed=42):
    random.seed(seed)
    selected = random.sample(train_triplets, k)

    examples = []
    for ex in selected:
        sentence = ex["sentence"].strip()
        triplets = ex.get("triplets", [])

        # Get unique entities from the triplets
        entity_set = set()
        for t in triplets:
            entity_set.add(t[0])
            entity_set.add(t[2])
        entities = ", ".join(sorted(entity_set))

        triplet_lines = "\n".join([f"({t[0]}, {t[1]}, {t[2]})" for t in triplets])

        examples.append(
            f"""Example:
Sentence: "{sentence}"
Entities: {entities}
Triplets:
{triplet_lines}
"""
        )

    return "\n".join(examples)

few_shot_prompt_2 = create_few_shot_prompt(train_triplets, k=2)

# Then use in your prompting like:
few_shot_prompt_2_examples = f"{few_shot_prompt_2}"
print(few_shot_prompt_2_examples)

In [ ]:
import random

def create_few_shot_prompt(train_triplets, k=5, seed=40):
    random.seed(seed)
    selected = random.sample(train_triplets, k)

    examples = []
    for ex in selected:
        sentence = ex["sentence"].strip()
        triplets = ex.get("triplets", [])

        # Get unique entities from the triplets
        entity_set = set()
        for t in triplets:
            entity_set.add(t[0])
            entity_set.add(t[2])
        entities = ", ".join(sorted(entity_set))

        triplet_lines = "\n".join([f"({t[0]}, {t[1]}, {t[2]})" for t in triplets])

        examples.append(
            f"""Example:
Sentence: "{sentence}"
Entities: {entities}
Triplets:
{triplet_lines}
"""
        )

    return "\n".join(examples)

few_shot_prompt_5 = create_few_shot_prompt(train_triplets, k=5)

# Then use in your prompting like:
few_shot_prompt_5_examples = f"{few_shot_prompt_5}"
print(few_shot_prompt_5_examples)

In [ ]:
import random

def create_few_shot_prompt(train_triplets, k=10, seed=10):
    random.seed(seed)
    selected = random.sample(train_triplets, k)

    examples = []
    for ex in selected:
        sentence = ex["sentence"].strip()
        triplets = ex.get("triplets", [])

        # Get unique entities from the triplets
        entity_set = set()
        for t in triplets:
            entity_set.add(t[0])
            entity_set.add(t[2])
        entities = ", ".join(sorted(entity_set))

        triplet_lines = "\n".join([f"({t[0]}, {t[1]}, {t[2]})" for t in triplets])

        examples.append(
            f"""Example:
Sentence: "{sentence}"
Entities: {entities}
Triplets:
{triplet_lines}
"""
        )

    return "\n".join(examples)

few_shot_prompt_10 = create_few_shot_prompt(train_triplets, k=10)

# Then use in your prompting like:
few_shot_prompt_10_examples = f"{few_shot_prompt_10}"
print(few_shot_prompt_10_examples)

In [ ]:
import random

def create_few_shot_prompt(train_triplets, k=20, seed=20):
    random.seed(seed)
    selected = random.sample(train_triplets, k)

    examples = []
    for ex in selected:
        sentence = ex["sentence"].strip()
        triplets = ex.get("triplets", [])

        # Get unique entities from the triplets
        entity_set = set()
        for t in triplets:
            entity_set.add(t[0])
            entity_set.add(t[2])
        entities = ", ".join(sorted(entity_set))

        triplet_lines = "\n".join([f"({t[0]}, {t[1]}, {t[2]})" for t in triplets])

        examples.append(
            f"""Example:
Sentence: "{sentence}"
Entities: {entities}
Triplets:
{triplet_lines}
"""
        )

    return "\n".join(examples)

few_shot_prompt_20 = create_few_shot_prompt(train_triplets, k=20)

# Then use in your prompting like:
few_shot_prompt_20_examples = f"{few_shot_prompt_20}"
print(few_shot_prompt_20_examples)

# Pair Level Model with all prompting techniques

In [ ]:
import os
import json
import time
from google import genai


GEMINI_API_KEY = ''
assert GEMINI_API_KEY, ""
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
client = genai.Client()

# Output holder
#all_responses = []

# Prompt styles
interaction_types = """
- mechanism: Pharmacokinetic interactions affecting drug metabolism
- effect: Pharmacodynamic effects or clinical outcomes
- advice: Recommendations, cautions, or contraindications
- int: General interaction mentions without specifics
- no interaction: Explicit statements of no interaction
"""

few_shot_examples = f"""
{interaction_types}
"""

# Inference loop
for i in range(8878, len(pairwise_df_records)):
    example = pairwise_df_records[i]
    sentence = example["sentence"]
    entity1 = example["entity1"]
    entity2 = example["entity2"]
    ground_truth = example["ground_truth"]

    output_entry = {
        "index": i,
        "sentence": sentence,
        "entity1": entity1,
        "entity2": entity2,
        "ground_truth_label":ground_truth
    }

    # Prepare prompts
    zero_shot_prompt = f"""Analyze drug interactions in the text below. Extract EXACTLY ONE label per drug pair using these interaction types:
- mechanism: Pharmacokinetic interactions affecting drug metabolism
- effect: Pharmacodynamic effects or clinical outcomes
- advice: Recommendations, cautions, or contraindications
- int: General interaction mentions without specifics
- no interaction: Explicit statements of no interaction

Rules:
1. For cautionary statements like "Caution is advised", always use "advice"
2. Output only one triplet per unique drug pair
3. Use normalized entity names from the text

Text: "{sentence}"
Entities are given: {entity1} and {entity2}

Answer with only label name. Do not include any additional text.
Extract EXACTLY ONE label per drug pair using these interaction types
Label:"""

    few_shot_prompt_2 = f"""Classify drug interactions using these examples:
    
    {few_shot_prompt_2_examples}

Now classify interactions in this new text: "{sentence}"
Entities are given: {entity1} and {entity2}
Output ONLY ONE label name for given pair out of these classes: mechanism, effect, advice, int, no interaction.
Label:"""
    
    few_shot_prompt_5 = f"""Classify drug interactions using these examples:
    
    {few_shot_prompt_5_examples}

Now classify interactions in this new text: "{sentence}"
Entities are given: {entity1} and {entity2}
Output ONLY ONE label name for given pair out of these classes: mechanism, effect, advice, int, no interaction.
Label:"""


    few_shot_prompt_10 = f"""Classify drug interactions using these examples:
    
    {few_shot_prompt_10_examples}

Now classify interactions in this new text: "{sentence}"
Entities are given: {entity1} and {entity2}
Output ONLY ONE label name for given pair out of these classes: mechanism, effect, advice, int, no interaction.
Label:"""


    few_shot_prompt_20 = f"""Classify drug interactions using these examples:
    
    {few_shot_prompt_20_examples}

Now classify interactions in this new text: "{sentence}"
Entities are given: {entity1} and {entity2}
Output ONLY ONE label name for given pair out of these classes: mechanism, effect, advice, int, no interaction.
Label:"""



    cot_prompt = f"""Analyze drug interactions step by step:

1. Identify given drug entities
2. For given drug pair:
   a. Check if text explicitly says "no interaction" → "no interaction"
   b. Check for caution words: "caution", "advised", "warning" → "advice"
   c. Check for metabolic terms: "metabolism", "inhibits", "induces" → "mechanism"
   d. Check for clinical outcomes: "caused", "resulted in", "increased" → "effect"
   e. Generic "interaction" mention → "int"
3. If multiple signals, choose the strongest:
   advice > mechanism > effect > int > no interaction
4. Output one triplet: (Entity1, interaction_type, Entity2)

Text: "Caution is advised when beginning, discontinuing, or changing the dose of DIAMOX in patients receiving primidone."

Step-by-step reasoning:
1. Entities: DIAMOX, primidone
2. "Caution is advised" → advice
3. No stronger signals
4. Output: advice

Now analyze: "{sentence}"
Now classify interactions in this new text: "{sentence}"
Entities are given: {entity1} and {entity2}
Output ONLY ONE label name for given pair out of these classes: mechanism, effect, advice, int, no interaction.
Reasoning Steps:[Gemma's Reasoning]
Answer with only label name. Do not include any additional text.
Label:
"""

    prompts = {
        "zero_shot_output": zero_shot_prompt,
        "few_shot_output_2": few_shot_prompt_2,
        "few_shot_output_5": few_shot_prompt_5,
        "few_shot_output_10": few_shot_prompt_10,
        "few_shot_output_20": few_shot_prompt_20,
        "cot_output": cot_prompt
    }

    for method_name, prompt in prompts.items():
        try:
            response = client.models.generate_content(
                model="gemma-3-27b-it",
                contents=prompt,
                config=genai.types.GenerateContentConfig(
                    temperature=0.0
                )
            )
            time.sleep(3.5)

            output_entry[method_name] = response.text.strip()
            print(f"[{method_name}] Finished index {i}")
            print(f"{method_name}: {response.text.strip()}")

        except Exception as e:
            print(f"Error at index {i} with method {method_name}: {e}")
            output_entry[method_name] = "ERROR"

    all_responses.append(output_entry)

    # Save after each example (optional safety)
    with open("pairwise_predictions.json", "w") as f:
        json.dump(all_responses, f, indent=2)